Cross Validation of Prophet Forecasting Nuitees dans l'hôtellerie Pays de la Loire 
Datasets https://github.com/jinnyto/touristcast/tree/datasets/datasets

In [1]:
import pandas as pd
import numpy as np
import datetime 
import matplotlib.pyplot as plt
import pyramid
%matplotlib inline
from statsmodels.tsa.seasonal import seasonal_decompose
import holidays
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
import fbprophet
def seasonal_MASE(truth, forecast, seasonality=1):    
    period = truth.shape[0] # T
    # print(period)
    forecast_errors = np.abs(truth - forecast)
    # print(forecast_errors)
    mean_absolute_forecast_error = np.sum(forecast_errors) / period
    # print(mean_absolute_forecast_error)

    naive_period = truth.shape[0] - seasonality # T - m
    # print(naive_period)
    # print(truth[seasonality:])
    # print(truth[:period - seasonality])
    naive_errors = np.abs(truth[seasonality:] - truth[:period - seasonality])
    mean_absolute_naive_error = np.sum(naive_errors) / naive_period
    
    return mean_absolute_forecast_error / mean_absolute_naive_error

In [2]:
dn = pd.read_csv('./datasets/nights_2010-2017.csv',parse_dates=['date'], index_col='date',usecols=[*range(1, 15)])

In [4]:
smase_fold = []
startyear = '2010-01-01'
for index in range(2011,2018):
    ld = []
    for year in range(2010,index):
        for date, name in sorted(holidays.FRA(years=year).items()):
            ld.append(date.strftime('%Y-%m-%d'))
    nextyear = str(index)+'-01-01'
    predictyear = str(index+1)+'-01-01'
    dftrain = dn[(dn.index>=startyear)&(dn.index<nextyear)]
    train_nights = np.array(dftrain.nights_PDL)
    dftest = dn[(dn.index>=nextyear)&(dn.index<predictyear)]
    test_nights = np.array(dftest.nights_PDL)
    fb = pd.DataFrame()
    fb['ds'] = dftrain.index
    fb['y'] = train_nights
    holidays_exo = pd.DataFrame({  
       'holiday': 'france',
       'ds': pd.to_datetime(ld),
    })
    fb_prophet = fbprophet.Prophet(yearly_seasonality = True, seasonality_mode='multiplicative', seasonality_prior_scale=0.04, changepoint_prior_scale=0.1, holidays=holidays_exo, holidays_prior_scale=0.01)
    fb_prophet.fit(fb)
    fb_forecast = fb_prophet.make_future_dataframe(periods=12, freq='M')
    fb_forecast = fb_prophet.predict(fb_forecast)
    tc_future_forecast= np.array(fb_forecast.tail(12).yhat)
    smase_fold.append(seasonal_MASE(test_nights,tc_future_forecast))


INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 8.0.
/home/aa/anaconda3/lib/python3.6/site-packages/fbprophet/forecaster.py:353: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  np.linspace(0, hist_size - 1, self.n_changepoints + 1)
/home/aa/anaconda3/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling d

In [5]:
np.sum(smase_fold) / len(smase_fold)

0.5037957706186224